In [4]:
from selenium import webdriver
import chromedriver_binary
import csv
from bs4 import BeautifulSoup
import time

In [2]:

def my_url(search):
    temp = 'https://www.cardekho.com/used-cars+in+{}'
    search = search.replace(' ', '-')
    url = temp.format(search)
    return url


def extract_records(obj, city):

    parent = obj.find('div', {'class': 'gsc_col-xs-7 carsName'})
    year_companyName_Name = (parent.find('a').text).split(" ")
    year = year_companyName_Name[0]
    companyName = year_companyName_Name[1]
    Name = ""
    for j in range(2, len(year_companyName_Name)):

        Name += year_companyName_Name[j] + " "

    price = parent.find('span', 'amnt').text

    k = obj.find('div', {'class': 'truncate dotlist-2'})
    i = 0
    for t in k:
        if i == 0:
            kms = t.text
        if i == 1:
            fuel = t.text
        if i == 2:
            transmission = t.text
        i = i+1

    kms = kms.replace(',', '')
    kms = kms.replace(' kms', '')

    #print(year,"//",companyName,"//",Name, "//", price, "//", kms,"//",fuel,"//",transmission)

    results = (Name, price, city, fuel, transmission, kms, year, companyName)
    return results


def main(search):
    driver = webdriver.Chrome()

    records = []
    url = my_url(search)

    driver.get(url)
    time.sleep(2)
    scroll_pause_time = 3
    screen_height = driver.execute_script("return window.screen.height;")   # get the screen height of the web
    i = 1
    while True:
    # scroll one screen height each time
        driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))  
        i += 1
        time.sleep(scroll_pause_time)
        scroll_height = driver.execute_script("return document.body.scrollHeight;")  
        if (screen_height) * i > scroll_height:
            break

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    soup_results = soup.find_all('div', {'class': 'shadowWPadding posR'})

    for item in soup_results:
        record = extract_records(item, search)
        if record:
            records.append(record)

    n = 1
    if search == "new delhi":
        n = 0
    writeRecords(records, n)


def writeRecords(records, num):
    with open('results.csv', 'a', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        if num == 0:
            writer.writerow(['Name', 'Price', 'Location', 'Fuel Type',
                            'Transmission', 'Kms Driven', 'Year', 'Company Name'])
        writer.writerows(records)


In [ ]:
cities = ['pune','new delhi','bangaluru','hyderabad','chennai','mumbai']
for city in cities:
    main(city)